In [22]:
from google.colab import files
uploaded_1 = files.upload()
uploaded_2 = files.upload()

Saving poker-hand-training-true.csv to poker-hand-training-true (1).csv


Saving poker-hand-testing.csv to poker-hand-testing (3).csv


In [147]:
import pandas as pd
import io
train = pd.read_csv(io.StringIO(uploaded_1['poker-hand-training-true.csv'].decode('utf-8')), header=None)
test = pd.read_csv(io.StringIO(uploaded_2['poker-hand-testing.csv'].decode('utf-8')), header=None)
train.head()
test.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1,1,1,13,2,4,2,3,1,12,0
1,3,12,3,2,3,11,4,5,2,5,1
2,1,9,4,6,1,4,3,2,3,9,1
3,1,4,3,13,2,13,2,1,3,6,1
4,3,10,2,7,1,2,2,11,4,9,0


In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation

In [0]:
def build_multilayer_perceptron():
    model = Sequential()
    model.add(Dense(128), input_shape=(10, ))
    model.add(Activation('relu'))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(10))
    model.add(Activation('sigmoid'))
    return model

In [0]:
model = build_multilayer_perceptron()

In [148]:
from sklearn import preprocessing
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import normalize

train_Y = train[:][10]
test_Y = test[:][10]

X_train, X_test, Y_train, Y_test = train_test_split(train, train_Y,train_size=0.7)


X_train.drop(10,axis=1,inplace=True)
X_test.drop(10,axis=1,inplace=True)
X_train = normalize(X_train)
X_test = normalize(X_test)



print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

(17507, 10) (17507,) (7503, 10) (7503,)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [0]:
clf = xgb.XGBClassifier()

In [0]:
clf_cv = GridSearchCV(clf, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)

In [152]:
print(X_train.shape,Y_train.shape)
clf_cv.fit(X_train, Y_train,verbose=1)

(17507, 10) (17507,)
Fitting 3 folds for each of 9 candidates, totalling 27 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  4.5min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [2, 4, 6],
                         'n_estimators': [50, 100, 200]},
             pre_dispatch='2*n_jobs', refit=True, retur

In [155]:
# 改めて最適パラメータで学習
clf = xgb.XGBClassifier(**clf_cv.best_params_)
clf.fit(X_train, Y_train,verbose=1)

# 学習モデルの保存、読み込み
# import pickle
# pickle.dump(clf, open("model.pkl", "wb"))
# clf = pickle.load(open("model.pkl", "rb"))

# 学習モデルの評価
pred = clf.predict(X_test)
print (confusion_matrix(Y_test, pred))
print (classification_report(Y_test, pred))

[[3010  717    1    0    0    0    0    0    0]
 [1497 1683   10    1    0    0    0    0    0]
 [  62  285   13    5    0    0    0    0    0]
 [  29  115    7    1    0    0    0    0    0]
 [   0   37    0    0    1    0    0    0    1]
 [  12    3    0    0    0    0    0    0    0]
 [   0   10    0    0    0    0    0    0    0]
 [   0    1    0    1    0    0    0    0    0]
 [   0    1    0    0    0    0    0    0    0]]
              precision    recall  f1-score   support

           0       0.65      0.81      0.72      3728
           1       0.59      0.53      0.56      3191
           2       0.42      0.04      0.07       365
           3       0.12      0.01      0.01       152
           4       1.00      0.03      0.05        39
           5       0.00      0.00      0.00        15
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00         2
           9       0.00      0.00      0.00         1

    accuracy             

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [157]:
from sklearn.ensemble import RandomForestClassifier
RFCmodel = RandomForestClassifier(n_estimators=100,random_state=0)
RFCmodel.fit(X_train,Y_train)
pred = RFCmodel.predict(X_test)
print(RFCmodel.score(X_test,Y_test))

0.6192189790750366
